In [ ]:
pip install neptune-notebooks

После получения новых датасетов запустим тренировку нашей модели. Будем тренировать ее на протяжении 25 эпох при помощи gpu, записывая логи в нептун. 

In [3]:
!python ../run_trivec.py --metrics_separately --epoch 25 --log --gpu --neptune_project vlasova.elizaveta/test

https://app.neptune.ai/vlasova.elizaveta/test/e/TES-9
Use device: cuda
Train

100% 25/25 [09:06<00:00, 21.85s/it]
Test


Теперь протестируем модель отдельно на тех данных, которые она уже видела и на новых для нее. Для этого я написала отдельный скрипт, который по уже натренированной модели проводит процедуру тестирования. Можно видеть, что все три метрики (roc/auc score, average presicion score, average presition at 50) сильно хуже для новых леккартсв, чем для уже известных.

In [6]:
!python ../test_trivec.py --metrics_separately --model_path=new_data/trivec_saved/TriVec/11_04_2021/24_15_24_06.pt

Use device: cpu
Test
Metric auc_roc is 0.9714980530107913 for seen test data
Metric auprc is 0.9495577531452466 for seen test data
Metric ap50 is 0.9667610921172114 for seen test data
Metric auc_roc is 0.3897883618620548 for new test data
Metric auprc is 0.44225102479351147 for new test data
Metric ap50 is 0.19069889743992313 for new test data


Такое поведение связано с тем, что модель основана на тренировке эмбеддингов, то есть представлений вершин и ребер нашего графа. При создании эмбеддингов мы указываем, что размер словаря для вершин равен количеству уникальных элементов во всем нашем датасете, а сам слой заполняется рандомными весами из равномерного распределения. Тем не менее, если при тренировке модели какие-то вершины совсем не задействуются, их эмбединги и не тренируются, то есть остаются случайными. Из-за этого при попытке что-то предсказать на таких вершинах, мы получаем по сути рандомные значения. Это и видно из полученных метрик.

